In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time
from tqdm import tqdm
import sys
import os

# models 디렉토리 추가
sys.path.append('./models')
from resnet import resnet18
from shake_pyramidnet import ShakePyramidNet

# CUDA 메모리 관리 최적화 설정
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.cuda.empty_cache()

# 디바이스 설정
force_cpu = False
device = torch.device("cpu") if force_cpu else torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU 수: {torch.cuda.device_count()}")
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
    print(f"가용 CUDA 메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# 테스트 데이터셋 설정 (CIFAR-100)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

testset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(
    testset, 
    batch_size=128,  # 메모리 사용량을 고려하여 배치 크기 조정
    shuffle=False, 
    pin_memory=(device.type == 'cuda'),
    num_workers=32
)

# 배치 수 계산하여 출력
total_batches = len(testloader)
print(f"총 테스트 배치 수: {total_batches}, 배치 크기: {testloader.batch_size}")
print(f"총 테스트 샘플 수: {len(testset)}")

# 모델 생성 함수
def create_models():
    # ResNet18 모델 생성
    model_resnet = resnet18()
    
    # ShakePyramidNet 모델 생성
    model_shake = ShakePyramidNet(
        depth=110,
        alpha=270,
        label=100  # CIFAR-100
    )
    
    return model_resnet, model_shake

# 모델 가중치 로드 함수
def load_model_weights(model, model_path, model_name):
    print(f"[{time.strftime('%H:%M:%S')}] {model_name} 가중치 로딩 중...")
    
    try:
        if device.type == 'cuda':
            state_dict = torch.load(model_path, map_location=device)
        else:
            state_dict = torch.load(model_path, map_location='cpu')
        
        # 'module.' 접두사 처리
        if all(k.startswith('module.') for k in state_dict.keys()):
            # GPU 사용 가능하고 여러 개일 경우 DataParallel 사용
            if device.type == 'cuda' and torch.cuda.device_count() > 1 and not force_cpu:
                model = nn.DataParallel(model)
            else:
                # CPU 모드이거나 GPU가 하나일 경우 접두사 제거
                state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
        
        # 가중치 로드
        model.load_state_dict(state_dict)
        print(f"[{time.strftime('%H:%M:%S')}] {model_name} 가중치 로딩 완료")
        
        # 디바이스로 모델 이동
        model = model.to(device)
        model.eval()
        
        return model
    
    except Exception as e:
        print(f"[{time.strftime('%H:%M:%S')}] {model_name} 가중치 로딩 중 오류 발생: {e}")
        sys.exit(1)

# 앙상블 평가 함수
def evaluate_ensemble(model_resnet, model_shake, weights=(0.45, 0.55)):
    """
    두 모델의 앙상블 평가 함수
    weights: (resnet_weight, shake_weight) - 두 모델의 가중치 (합이 1이 되어야 함)
    """
    try:
        print(f"\n[{time.strftime('%H:%M:%S')}] 앙상블 모델 평가 시작...")
        print(f"앙상블 가중치: ResNet18 = {weights[0]:.2f}, ShakePyramidNet = {weights[1]:.2f}")
        
        # 가중치 합이 1인지 확인 및 정규화
        weights = tuple(w / sum(weights) for w in weights)
        
        correct_top1 = 0
        correct_top5 = 0
        total = 0
        
        start_time = time.time()
        
        with torch.no_grad():
            pbar = tqdm(testloader, desc="앙상블 평가 중", unit="batch")
            for inputs, labels in pbar:
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    
                    # ResNet18 예측
                    outputs_resnet = model_resnet(inputs)
                    probs_resnet = torch.softmax(outputs_resnet, dim=1)
                    
                    # ShakePyramidNet 예측
                    outputs_shake = model_shake(inputs)
                    probs_shake = torch.softmax(outputs_shake, dim=1)
                    
                    # 가중 앙상블 (소프트 보팅)
                    ensemble_probs = weights[0] * probs_resnet + weights[1] * probs_shake
                    
                    # Top-1 정확도
                    _, predicted = ensemble_probs.max(1)
                    batch_correct = predicted.eq(labels).sum().item()
                    total += labels.size(0)
                    correct_top1 += batch_correct
                    
                    # Top-5 정확도
                    _, top5_idx = ensemble_probs.topk(5, 1, largest=True, sorted=True)
                    batch_correct_top5 = top5_idx.eq(labels.view(-1, 1).expand_as(top5_idx)).sum().item()
                    correct_top5 += batch_correct_top5
                    
                    # 진행 바 업데이트
                    current_acc = 100. * correct_top1 / total
                    pbar.set_postfix({"앙상블 정확도": f"{current_acc:.2f}%"})
                
                except RuntimeError as e:
                    print(f"\n오류 발생: {e}")
                    if device.type == 'cuda':
                        print("CPU로 전환하여 계속합니다...")
                        inputs, labels = inputs.cpu(), labels.cpu()
                        model_resnet = model_resnet.cpu()
                        model_shake = model_shake.cpu()
                        
                        outputs_resnet = model_resnet(inputs)
                        probs_resnet = torch.softmax(outputs_resnet, dim=1)
                        
                        outputs_shake = model_shake(inputs)
                        probs_shake = torch.softmax(outputs_shake, dim=1)
                        
                        ensemble_probs = weights[0] * probs_resnet + weights[1] * probs_shake
                        
                        # Top-1 정확도
                        _, predicted = ensemble_probs.max(1)
                        batch_correct = predicted.eq(labels).sum().item()
                        total += labels.size(0)
                        correct_top1 += batch_correct
                        
                        # Top-5 정확도
                        _, top5_idx = ensemble_probs.topk(5, 1, largest=True, sorted=True)
                        batch_correct_top5 = top5_idx.eq(labels.view(-1, 1).expand_as(top5_idx)).sum().item()
                        correct_top5 += batch_correct_top5
                        
                        # 다시 GPU로 이동
                        model_resnet = model_resnet.to(device)
                        model_shake = model_shake.to(device)
        
        accuracy_top1 = 100.0 * correct_top1 / total
        accuracy_top5 = 100.0 * correct_top5 / total
        
        evaluation_time = time.time() - start_time
        
        print(f'\n[{time.strftime("%H:%M:%S")}] 앙상블 모델 평가 완료:')
        print(f'- 앙상블 Top-1 정확도: {accuracy_top1:.2f}%')
        print(f'- 앙상블 Top-5 정확도: {accuracy_top5:.2f}%')
        print(f'- 평가 소요 시간: {evaluation_time:.2f}초')
        
        return accuracy_top1, accuracy_top5
    
    except Exception as e:
        print(f"[{time.strftime('%H:%M:%S')}] 앙상블 평가 중 예외 발생: {e}")
        return 0.0, 0.0

# 여러 가중치 조합 시도 함수
def try_different_weights(model_resnet, model_shake):
    """여러 가중치 조합을 시도하여 최적의 앙상블 찾기"""
    print(f"\n[{time.strftime('%H:%M:%S')}] 다양한 앙상블 가중치 조합 시도...")
    
    # 시도할 가중치 조합들
    weight_combinations = [
        (0.5, 0.5),       # 동일 가중치
        (0.45, 0.55),     # 성능 차이(2%)를 반영한 가중치
        (0.4, 0.6),       # 더 차이를 둔 가중치
        (0.3, 0.7),       # 큰 차이를 둔 가중치
        (0, 1.0),         # ShakePyramidNet만 사용
        (1.0, 0)          # ResNet18만 사용
    ]
    
    results = []
    
    for w_resnet, w_shake in weight_combinations:
        print(f"\n가중치 조합 시도: ResNet18 = {w_resnet:.2f}, ShakePyramidNet = {w_shake:.2f}")
        accuracy, _ = evaluate_ensemble(model_resnet, model_shake, weights=(w_resnet, w_shake))
        results.append((w_resnet, w_shake, accuracy))
        
        # GPU 메모리 정리
        torch.cuda.empty_cache()
    
    # 결과 정렬 및 출력
    results.sort(key=lambda x: x[2], reverse=True)
    
    print("\n------- 가중치 조합 결과 (정확도 순) -------")
    for w_resnet, w_shake, accuracy in results:
        print(f"ResNet18 = {w_resnet:.2f}, ShakePyramidNet = {w_shake:.2f}: {accuracy:.2f}%")
    
    best_weights = (results[0][0], results[0][1])
    best_accuracy = results[0][2]
    
    print(f"\n최적 가중치 조합: ResNet18 = {best_weights[0]:.2f}, ShakePyramidNet = {best_weights[1]:.2f}")
    print(f"최고 정확도: {best_accuracy:.2f}%")
    
    return best_weights, best_accuracy

if __name__ == "__main__":
    # 파일 및 디렉토리 존재 확인
    if not os.path.exists('models/resnet.py') or not os.path.exists('models/shake_pyramidnet.py'):
        print("오류: models 디렉토리에 필요한 모델 파일이 없습니다.")
        sys.exit(1)
        
    if not os.path.exists('best_model_resnet18.pth'):
        print("오류: best_model_resnet18.pth 파일을 찾을 수 없습니다.")
        sys.exit(1)
        
    if not os.path.exists('best_model_shake_pyramidnet.pth'):
        print("오류: best_model_shake_pyramidnet.pth 파일을 찾을 수 없습니다.")
        sys.exit(1)
    
    # 모델 생성
    print(f"\n[{time.strftime('%H:%M:%S')}] 모델 생성 중...")
    model_resnet, model_shake = create_models()
    
    # 모델 가중치 로드
    model_resnet = load_model_weights(
        model_resnet, 
        'best_model_resnet18.pth', 
        'ResNet18'
    )
    
    model_shake = load_model_weights(
        model_shake, 
        'best_model_shake_pyramidnet.pth', 
        'ShakePyramidNet'
    )
    
    # 알려진 정확도 기반 가중치 (이미 알고 있는 정보 활용)
    # ResNet18: 80%, ShakePyramidNet: 82%
    known_resnet_acc = 80.0
    known_shake_acc = 82.0
    
    # 단순 비율 기반 가중치 계산
    total_acc = known_resnet_acc + known_shake_acc
    w_resnet = known_resnet_acc / total_acc  # 약 0.494
    w_shake = known_shake_acc / total_acc    # 약 0.506
    
    # 성능 차이를 더 강조한 가중치
    w_resnet = 0.45  # 더 낮은 가중치 부여
    w_shake = 0.55   # 더 높은 가중치 부여
    
    weights = (w_resnet, w_shake)
    
    print(f"\n[{time.strftime('%H:%M:%S')}] 앙상블 성능 평가 시작")
    print(f"기본 가중치: ResNet18 = {weights[0]:.2f}, ShakePyramidNet = {weights[1]:.2f}")
    
    # 앙상블 평가
    ensemble_acc_top1, ensemble_acc_top5 = evaluate_ensemble(model_resnet, model_shake, weights)
    
    # 메모리 정리
    torch.cuda.empty_cache()
    
    # 여러 가중치 조합 시도 (선택적)
    try_different_combinations = True
    
    if try_different_combinations:
        print("\n------- 다양한 가중치 조합 시도 -------")
        best_weights, best_accuracy = try_different_weights(model_resnet, model_shake)
        
        # 알려진 모델 정확도와 비교
        print("\n------- 최종 성능 비교 -------")
        print(f"ResNet18 알려진 정확도: {known_resnet_acc:.2f}%")
        print(f"ShakePyramidNet 알려진 정확도: {known_shake_acc:.2f}%")
        print(f"최적 앙상블 정확도: {best_accuracy:.2f}%")
        
        improvement_over_resnet = best_accuracy - known_resnet_acc
        improvement_over_shake = best_accuracy - known_shake_acc
        
        print(f"\nResNet18 대비 향상: {improvement_over_resnet:.2f}%")
        print(f"ShakePyramidNet 대비 향상: {improvement_over_shake:.2f}%")
    else:
        # 기본 앙상블 결과만 표시
        print("\n------- 최종 성능 비교 -------")
        print(f"ResNet18 알려진 정확도: {known_resnet_acc:.2f}%")
        print(f"ShakePyramidNet 알려진 정확도: {known_shake_acc:.2f}%")
        print(f"앙상블 정확도: {ensemble_acc_top1:.2f}%")
        
        improvement_over_resnet = ensemble_acc_top1 - known_resnet_acc
        improvement_over_shake = ensemble_acc_top1 - known_shake_acc
        
        print(f"\nResNet18 대비 향상: {improvement_over_resnet:.2f}%")
        print(f"ShakePyramidNet 대비 향상: {improvement_over_shake:.2f}%")

Using device: cuda
GPU 수: 2
GPU 이름: NVIDIA RTX A5000
가용 CUDA 메모리: 25.43 GB
Files already downloaded and verified
총 테스트 배치 수: 79, 배치 크기: 128
총 테스트 샘플 수: 10000

[12:25:19] 모델 생성 중...
[12:25:20] ResNet18 가중치 로딩 중...
[12:25:20] ResNet18 가중치 로딩 완료
[12:25:20] ShakePyramidNet 가중치 로딩 중...
[12:25:20] ShakePyramidNet 가중치 로딩 완료

[12:25:20] 앙상블 성능 평가 시작
기본 가중치: ResNet18 = 0.45, ShakePyramidNet = 0.55

[12:25:20] 앙상블 모델 평가 시작...
앙상블 가중치: ResNet18 = 0.45, ShakePyramidNet = 0.55


앙상블 평가 중:   0%|                                                                                     | 0/79 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a CuDNNError: cuDNN error: CUDNN_STATUS_BAD_PARAM
Exception raised from run_conv_plan at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:374 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x7f19f6003897 in /usr/local/lib/python3.11/dist-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0xe1c861 (0x7f19f6ea5861 in /usr/local/lib/python3.11/dist-packages/torch/lib/libtorch_cuda.so)
frame #2: <unknown function> + 0x1095d83 (0x7f19f711ed83 in /usr/local/lib/python3.11/dist-packages/torch/lib/libtorch_cuda.so)
frame #3: <unknown function> + 0x1097c2c (0x7f19f7120c2c in /usr/local/lib/python3.11/dist-packages/torch/lib/libtorch_cuda.so)
frame #4: <unknown function> + 0x109817b (0x7f19f712117b in /usr/local/lib/python